In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
REPO_NAME = 'sewer-nfl'
import sys, os
cwd = str(os.getcwd())
repo_dir = cwd[:cwd.find(REPO_NAME)+len(REPO_NAME)]
sys.path.insert(0,repo_dir)

In [16]:
import pandas as pd
import nfl_data_py as nflreadr

In [15]:
from warehouse.utilities.elo import calculate_elo_metric

In [18]:
api_data = nflreadr.import_pbp_data(range(2021,2023))

2021 done.
2022 done.
Downcasting floats.


In [19]:
api_data = api_data[api_data['week']<=18]

In [49]:
from warehouse.pipelines.pbp.pbp import player_involvement, player_involved

In [57]:
player_involvement_df = player_involvement(pbp_data=api_data)

In [62]:
player_involved(player_involvement_df=player_involvement_df,
                play_id = 40.0,
                game_id = '2021_01_ARI_TEN',
                player_id = '00-0032560')

0